# Optical Flow

Optical flow is the pattern of apparent motion of image objects between two consecutive frames caused by the movement of object or camera. It is vector field where each vector denotes the movement of points from first frame to second. Generally, optical flow is used to track the motion of objects in a video. Lets assume you have captured two frames with a small time difference $\Delta t$. You wish to track the motion of a pixel located at $(x, y)$ in first frame. In the second frame, the pixel has moved to $(x + \Delta x, y + \Delta y)$. The vector $(\Delta x, \Delta y)$ is the optical flow vector. 

<!-- ![Optical Flow](../../images/blogs/Lucas-Kanade/BCC.png) -->

<!-- <img src="/images/blogs/Lucas-Kanade/BCC.png"
alt="Optical Flow" width="600"> -->

![Optical Flow : Brightness Constancy Constraint](../../images/blogs/Lucas-Kanade/BCC.png){style="display: block; margin: auto; width: 600px;"}




$$I(x, y, t) = I(x + \Delta x, y + \Delta y, t + \Delta t)$$

The above equation is the basic assumption in optical flow. It assumes that intensity of an object does not change between two consecutive frames. This constraint is called **brightness constancy constraint**. This equation can be expanded using taylor series to get the optical flow equation.

$$I(x + \Delta x, y + \Delta y, t + \Delta t) \approx I(x, y, t) + \frac{\partial I}{\partial x} \Delta x + \frac{\partial I}{\partial y} \Delta y + \frac{\partial I}{\partial t} \Delta t = I(x, y, t)$$

$$\frac{\partial I}{\partial x} \Delta x + \frac{\partial I}{\partial y} \Delta y + \frac{\partial I}{\partial t} \Delta t = 0$$

$$\frac{\partial I}{\partial x} u + \frac{\partial I}{\partial y} v + \frac{\partial I}{\partial t} = 0$$

where $u = \frac{\Delta x}{\Delta t}$ and $v = \frac{\Delta y}{\Delta t}$ are the optical flow velocities in x and y directions respectively. 

The above equation is called optical flow equation. It is a single equation with two unknowns $u$ and $v$. Hence, it is an ill-posed problem. To solve this problem, we need to make some more assumptions. One of the assumption to arrive here was brightness constancy assumption. It assumes that the intensity of an object does not change between two consecutive frames. The other assumption made was given by lucas and kanade. They assumed that the optical flow is same for all the pixels in a neighborhood. This assumption is called spatial coherence assumption.



# Lucas Kanade Optical Flow

The Lucas-Kanade method is a widely used differential method for optical flow estimation developed by Bruce D. Lucas and Takeo Kanade. It assumes that the flow is essentially constant in a local neighbourhood of the pixel under consideration, and solves the basic optical flow equations for all the pixels in that neighbourhood by the least squares criterion.

![Lucas Kanade : spatial coherence assumption](../../images/blogs/Lucas-Kanade/LK-constraint.png){style="display: block; margin: auto; width: 400px;"}



$$\frac{\partial I}{\partial x} u + \frac{\partial I}{\partial y} v + \frac{\partial I}{\partial t} = 0$$

$$E_x = \frac{\partial I}{\partial x}, E_y = \frac{\partial I}{\partial y}, E_t = \frac{\partial I}{\partial t}$$

$$E_x u + E_y v + E_t = 0$$

$$\begin{bmatrix} E_x & E_y \end{bmatrix} \begin{bmatrix} u \\ v \end{bmatrix} = -E_t$$

$$ \begin{bmatrix} E_x(i-1,j-1) & E_y(i-1,j-1) \\ E_x(i-1,j) & E_y(i-1,j) \\ E_x(i-1,j+1) & E_y(i-1,j+1) \\ E_x(i,j-1) & E_y(i,j-1) \\ E_x(i,j) & E_y(i,j) \\ E_x(i,j+1) & E_y(i,j+1) \\ E_x(i+1,j-1) & E_y(i+1,j-1) \\ E_x(i+1,j) & E_y(i+1,j) \\ E_x(i+1,j+1) & E_y(i+1,j+1) \end{bmatrix} \begin{bmatrix} u \\ v \end{bmatrix} = \begin{bmatrix} -E_t(i-1,j-1) \\ -E_t(i-1,j) \\ -E_t(i-1,j+1) \\ -E_t(i,j-1) \\ -E_t(i,j) \\ -E_t(i,j+1) \\ -E_t(i+1,j-1) \\ -E_t(i+1,j) \\ -E_t(i+1,j+1) \end{bmatrix}$$

$$A_{(9,2)}  x_{(2,1)} = b_{(9,1)}$$

$$A \overrightarrow{x} = \overrightarrow{b}$$

$$x = (A^T A)^{-1} A^T b$$

So for a chosen patch we can estimate the optical flow using above equation. This could also be written as

$$\begin{bmatrix} u \\ v \end{bmatrix} = \begin{bmatrix} \sum E_x E_x & \sum E_x E_y \\ \sum E_y E_x & \sum E_y E_y \end{bmatrix}^{-1} \begin{bmatrix} -\sum E_x E_t \\ -\sum E_y E_t \end{bmatrix}$$

This is the final equation used to estimate the optical flow using Lucas Kanade method. The above equation is solved for each patch in the image to get the optical flow vectors. Optical flow algorithm could be summarized as follows:

1. Compute image gradients $E_x$ and $E_y$.
2. Compute temporal gradient $E_t$.
3. Compute the matrix $A$ and vector $b$ for each patch.
4. Solve the equation $A \overrightarrow{x} = \overrightarrow{b}$ to get the optical flow vectors.

# Lucas Kanade Implementation

Coming soon...!